# RESTful API and Web Service
This notebook will explore how to setup the actual LIMS service. We will also talk about the code behind the API.

## Running the Service
Prerequisites: You should have all dependencies (Flask) installed. This step is already done if you ran "pip install -r requirements.txt". You also need to have the database built and a Mongo instance running locally on port 27017. Steps to do all of this can be found [here](https://nbviewer.jupyter.org/github/rguo123/m2g-lims/blob/master/docs/Dataset_Registration.ipynb).

**Note**: If a Mongo Instance is not running, the service will not load!

#### 1. Run (python3) server.py in the [app directory](https://github.com/rguo123/m2g-lims/tree/master/app).
#### 2. Go to http://localhost:5000/
#### 3. Click on stuff!

### You should see this in your web browser:
<img src="./images/mvp_lims.png" alt="MVP" style="width: 60%;"/>


The rest of this notebook will now talk about the implementation of this actual service.

## RESTful API
The service is implemented as a RESTFul API. The client just requests the server for certain Mongo documents with JQuery and our Flask backend makes sure that a correct result is returned. We will focus mainly on the backend.

### Backend
Right now, the entire service is extremely simple. We do hope to add a lot more routes in the future for advanced search queries, filters, pipeline logging, etc. However, for now, there are only four main routes that we briefly discuss below.

#### Index Route ('/')
This route is what you get when you load the page. All it does is fetch all the dataset documents and returns them along with how many subjects is in each dataset.

**Note**: The status column you see on the index page is just a proof-of-concept for something we want to implement in the future. If there are missing derivatives or errors in a dataset, we would like to specify that. Since we aren't dealing with different pipelines and are just treating everything in m2g.io as absolute, we don't have a "error" field in the database yet.

In [ ]:
@app.route("/")
def index():
    cursor = lims.find({"subjects" : { "$exists": True}})
    datasets = []
    for doc in cursor:
        datasets.append({doc["_id"]: len(doc["subjects"])})
    return render_template("index.html", datasets = datasets)

#### LIMS ID Request Route ('/lims_id_request')
This route is the main driver for the entire service. The client will access this route with a GET Request and give either a particular dataset or subject query. This route will then build and return a URL that leads to the requested information. What this route expects from the client is a valid document id in the LIMS database.

In [ ]:
@app.route("/lims_id_request")
def lims_request():
    request_id = request.headers.get("id")
    if (request_id.find("sub-") != -1):
        return url_for("subject", subject_id = request_id)
    else:
        return url_for("dataset", dataset_id = request_id)

#### Dataset and Subject Route
These routes are very similar so we will combine them here. They both perform the actual request to the LIMS database. The only difference is that different HTML tables will get rendered since the information for a dataset document and subject document are very different.

In [ ]:
@app.route("/dataset")
def dataset():
    dataset_id = request.args.get("dataset_id")
    dataset = lims.find_one({"_id": dataset_id})
    subjects = dataset["subjects"]
    return render_template("dataset_table.html", dataset_id = dataset_id, subjects = subjects)

@app.route("/subject")
def subject():
    subject_id = request.args.get("subject_id")
    subject = lims.find_one({"_id": subject_id})
    # get rid of ID so easier to process derivatives
    subject.pop("_id")
    return render_template("subject_table.html", subject_id = subject_id, subject = subject)

Note that for the subject route, we have to pop the id field. This is because actually rendering all the nested dictionaries in the database is extremely annoying. For all the HTML tables starting with the index route, all the important information is parsed in using Jinja2 Templating. However, since subject documents have so much information, displaying everything is difficult. 